In [28]:
from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
from selenium.webdriver.common.keys import Keys
import json
import re
import pandas as pd
# driver.get(f"https://en.wikipedia.org/wiki/Category:Towns_in_{states[0]}")
# time.sleep(5)
# town_list = driver.find_elements(By.XPATH,'//*[@id="mw-pages"]/div/div')


In [38]:
def load_data(file):
    with open(file,"r",encoding="utf-8") as f:
        data = json.load(f)
    return data

def save_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
        
def clean_data(towns):
    towns = towns.split("\n")
    new_town_list = []
    for town in towns:
        
        if(len(town)>1):
            if("," in town):
                new_town_list.append(town.split(",")[0])
            elif("(" in town):
                index = town.index("(")
                new_town_list.append(town[0:index-1])
                
            else:
                new_town_list.append(town)
    return new_town_list

def get_towns_dict(driver,states):
    towns_dict = {}
    for state in states:
        driver.get(f"https://en.wikipedia.org/wiki/Category:Towns_in_{state.title()}")
        time.sleep(5)
        town_list = driver.find_elements(By.XPATH,'//*[@id="mw-pages"]/div/div')
        cleaned_town_list = clean_data(town_list[0].text)
        towns_dict[state] = cleaned_town_list
        
    return towns_dict



In [39]:
states = ["johor","kedah","kelantan","perak","selangor","malacca","negeri sembilan","pahang","perlis","penang","sabah","sarawak","terengganu"]
service = Service(executable_path=r"C:\Users\ZACK LEE\Documents\Coding\chromedriver_win32\chromedriver.exe")
op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(service=service)

towns_dict = get_towns_dict(driver,states)

In [40]:
save_data("data/towns_dict.json",towns_dict)

In [50]:
job_dict_malaysia = load_data("data/job_dict_malaysia.json")
job_dict_singapore = load_data("data/test_job_dict_linkedin.json")
job_dict = {}
job_dict.update(job_dict_malaysia)
job_dict.update(job_dict_singapore)

In [32]:
def get_separated_roles(job_dict):

    roles_dict = {}
    data_science = []
    data_engineer = []
    machine_learning = []
    analyst = []
    art_int = []
    # left_over= []
    # statistics= []
    # job_dict
    i = 0
    for listing,key in zip(job_dict,list(job_dict.keys())):
        i+=1
        inf = job_dict.get(key)
        role = inf.get('role')
        ds = bool(re.search(".*[Dd]ata\s.*[Ssc]ien.*", role))
        de = bool(re.search(".*[Dd]ata\s.*[Ee]ngine.*", role))
        ml_long = bool(re.search(".*[Mm]achine\s.*[Ll]earn.*",role))
        ml = bool(re.search(".*[Mm][Ll]\s.*", role))
        anal = bool(re.search(".*[Dd]ata\s.*[Aa]nal.*", role))
        ai_long = bool(re.search(".*[Aa]rtificial\s.*[Ii]ntelligence.*", role))
        ai = bool(re.search(".*\s[Aa][Ii]\s.*", role))
        # stat = bool(re.search(".*[Sst]at.*\s.*[Pp]rogram.*", role))
        if(ds):
            data_science.append(listing)
        elif(de):
            data_engineer.append(listing)
        elif(ml_long or ml):
            machine_learning.append(listing)
        elif(anal):
            analyst.append(listing)
        elif(ai or ai_long):
            art_int.append(listing)
        # elif(stat):
        #     statistics.append(listing)
        # else:
        #     left_over.append(listing)
    roles_dict = {"Data Analyst":analyst,"Machine Learning":machine_learning+art_int,"Data Scientist":data_science,"Data Engineer":data_engineer}
    return roles_dict

def init_column(df):
    roles = ['DS_count','DA_count','DE_count','ML_count']
    for role in roles:
        df[role] = 0
    return df

In [55]:
roles_dict = get_separated_roles(job_dict)

In [63]:
roles_towns_dict = {}
for role in roles_dict.keys():
    loc_array = []
    for link in roles_dict[role]:
        location = job_dict.get(link)['location']
        if(location != ""):
            loc_array.append(location)
    roles_towns_dict[role]=loc_array
        # print()
        # return

In [68]:
save_data("roles_towns_dict.json",roles_towns_dict)

In [96]:
roles_towns_dict = load_data("roles_towns_dict.json")
towns_dict = load_data("data/towns_dict.json")

In [97]:
federal = ['kuala lumpur','putrajaya','labuan','singapore']
all_locations = list(map(lambda x:x.lower(),towns_dict.keys()))
all_locations.extend(federal)

In [98]:
loc_df = pd.DataFrame(all_locations,columns = ["Words"])
loc_df = init_column(loc_df)

In [99]:
loc_df

,Words,DS_count,DA_count,DE_count,ML_count
0,johor,0,0,0,0
1,kedah,0,0,0,0
2,kelantan,0,0,0,0
3,perak,0,0,0,0
4,selangor,0,0,0,0
5,malacca,0,0,0,0
6,negeri sembilan,0,0,0,0
7,pahang,0,0,0,0
8,perlis,0,0,0,0
9,penang,0,0,0,0


In [105]:
def update_word_count(roles_dict,towns_dict,df):
    federal = ['kuala lumpur','putrajaya','labuan','singapore']
    states = ["johor","kedah","kelantan","perak","selangor","malacca","negeri sembilan","pahang","perlis","penang","sabah","sarawak","terengganu"]
    for key in roles_dict.keys():
        if(key=="Data Scientist"):
            column = "DS_count"
        elif(key=="Data Analyst"):
            column = "DA_count"
        elif(key=="Data Engineer"):
            column = "DE_count"
        else:
            column = "ML_count"
        count_list = [0]*len(df)
        for loc in roles_dict[key]:
            loc_name = ""
            if("," in loc):
                loc = loc.split(",")[0]
            elif("/" in loc):
                loc = loc.split("/")[0]
            if((loc.lower() in federal) or (loc.lower() in states)):
                loc_name = loc.lower()
#                 print(loc_name)
            else:
                for state in towns_dict.keys():
                    if(loc in towns_dict[state]):
                        loc_name = state
                        break
#             print("data: ",loc)
#             print(loc_name)
            if(loc_name != ""):
                index = df[df["Words"]==loc_name]['Words'].index[0]
                        # print(word, index)
                count_list[index]+=1
            df[column] = count_list
    

            

In [106]:
update_word_count(roles_towns_dict,towns_dict,loc_df)
loc_df

,Words,DS_count,DA_count,DE_count,ML_count
0,johor,1,1,0,0
1,kedah,0,0,0,0
2,kelantan,0,0,0,0
3,perak,0,2,0,0
4,selangor,27,38,20,5
5,malacca,0,3,0,0
6,negeri sembilan,1,0,0,0
7,pahang,2,1,0,0
8,perlis,0,0,0,0
9,penang,18,18,9,3


In [107]:
loc_df.to_csv("data/word_count_data/location.csv")

In [108]:
from sqlalchemy import create_engine
import pymysql
import json
def load_data(file):
    with open(file,"r",encoding="utf-8") as f:
        data = json.load(f)
    return data

db_details = load_data("data/db_details.json")
hostname=db_details["DBHOST"]
dbname=db_details["DBNAME"]
uname=db_details["DBUSER"]
pwd=db_details["DBPASS"]
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

In [109]:
loc_df.to_sql('location', engine, index=False)

17

In [110]:
loc_df

,Words,DS_count,DA_count,DE_count,ML_count
0,johor,1,1,0,0
1,kedah,0,0,0,0
2,kelantan,0,0,0,0
3,perak,0,2,0,0
4,selangor,27,38,20,5
5,malacca,0,3,0,0
6,negeri sembilan,1,0,0,0
7,pahang,2,1,0,0
8,perlis,0,0,0,0
9,penang,18,18,9,3
